## (0) Import Packages

In [1]:
import sys
sys.path.append('../src/') 
import numpy as np
import torch
import Core as C
import math
import pickle
import pandas as pd
import time

## (1) Load Data and Batch Data

In [2]:
# Unpickle file
with open('./Pickle_storage/test_data_tokenized_WA.pkl', 'rb') as handle:
    data, embed_map = pickle.load(handle) # We stored the data and embedding map in one file

# Visualize data
len(pd.DataFrame(data)['labels'][8])
pd.DataFrame(data).head()

,accession_id,aligned_seqs,scores,labels,numerical_tokens
0,EU100698,ATGAG-GACTATCA-T--TGCTTTGAGCTA-CATT-CTA-T-GTCT...,1169.0,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, ...","[0, 2, 3, 0, 3, 4, 3, 0, 1, 2, 0, 2, 1, 0, 4, ..."
1,EU199367,ATGA-AGACTATCA-T--TGCTTTGAGCTA-CATT-CTA-T-GTCT...,1169.0,"[0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, ...","[0, 2, 3, 0, 4, 0, 3, 0, 1, 2, 0, 2, 1, 0, 4, ..."
2,CY026603,TATTAACC--ATGAAGACTATCAT-TGCTTTGAGCTA-CATT-CTA...,1172.0,"[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, ...","[2, 0, 2, 2, 0, 0, 1, 1, 4, 4, 0, 2, 3, 0, 0, ..."
3,CY026771,TTCTATTAACC--ATGAAGACTATCAT-TGCTTTGAGCTA-CATT-...,1171.0,"[1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, ...","[2, 2, 1, 2, 0, 2, 2, 0, 0, 1, 1, 4, 4, 0, 2, ..."
4,EU516081,ATGAA-AG-TA-AAACTAC-TGGTCC-T--G-T-T-AT-GC-A-CA...,1171.0,"[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, ...","[0, 2, 3, 0, 0, 4, 0, 3, 4, 2, 0, 4, 0, 0, 0, ..."


In [3]:
# Create embedding and batch data
ds = C.dataset.Dataset(data, embed=embed_map)
dl = torch.utils.data.DataLoader(ds, batch_size=1, pin_memory=True)  # The data loader

# Train/Val Split
train_data, val_data = torch.utils.data.random_split(
    dl, [math.floor(len(dl)*0.8), math.floor(len(dl)*0.2)+1])

In [4]:
# Sandbox
#seq, label = next(iter(dl))
#label

## (2) Define Model

In [5]:
class TransformerModel(torch.nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super().__init__()
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = torch.nn.TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layers, nlayers)
        self.encoder = torch.nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = torch.nn.Linear(ninp, 2)
        self.softmax = torch.nn.Softmax(dim=2)

        self.init_weights()

    @staticmethod
    def generate_square_subsequent_mask(sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        output = self.softmax(output)
        return output


class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

## (3) Define Hyperparameters

In [6]:
ntokens = len(embed_map) # the size of vocabulary
emsize = 10 # embedding dimension
nhid = 10 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)

In [7]:
#Sandbox: Figure out loss function
#src_mask = model.generate_square_subsequent_mask(seq.size(0))
#output = model(seq, src_mask).data.view(-1,2)
#criterion(output,label.view(-1))

## (4) Define Functions for Training and Validation Steps

In [8]:
# Define train and validation functions
criterion = torch.nn.CrossEntropyLoss()
lr = 0.01 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()

    for batch, data in enumerate(train_data.dataset):
        seq, targets = data # embedded sequence, position labels
        optimizer.zero_grad()
        
        src_mask = model.generate_square_subsequent_mask(seq.size(0))
        output = model(seq, src_mask).view(-1,2)
        loss = criterion(output, targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d} batches | '
                  'ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch,
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    
    with torch.no_grad():
        for val_batched in data_source.dataset:
            seq, targets = val_batched
            src_mask = model.generate_square_subsequent_mask(seq.size(0))
            output = eval_model(seq, src_mask).view(-1,2)
            # Calculate loss: Might need a better way
            total_loss += criterion(output, targets.view(-1)).item()
    return total_loss / (len(data_source) -1)

## (5) Train Model

In [9]:
# Train model
best_val_loss = float(0)
epochs = 3 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} '
          .format(epoch, (time.time() - epoch_start_time),
                                     val_loss))
    print('-' * 80)

    if val_loss > best_val_loss:
        best_val_loss = val_loss
        best_model = model

| epoch   1 |   200 batches | ms/batch 32.59 | loss  0.68 | ppl     1.97
| epoch   1 |   400 batches | ms/batch 41.36 | loss  0.61 | ppl     1.84
| epoch   1 |   600 batches | ms/batch 28.20 | loss  0.58 | ppl     1.79
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 32.88s | valid loss  2.81 
--------------------------------------------------------------------------------
| epoch   2 |   200 batches | ms/batch 37.33 | loss  0.56 | ppl     1.76
| epoch   2 |   400 batches | ms/batch 26.64 | loss  0.56 | ppl     1.75
| epoch   2 |   600 batches | ms/batch 24.41 | loss  0.56 | ppl     1.75
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 27.15s | valid loss  2.80 
--------------------------------------------------------------------------------
| epoch   3 |   200 batches | ms/batch 24.20 | loss  0.56 | ppl     1.75
| epoch   3 |   400 batches | ms/batch 